In [ ]:
#############################################################################
# zlib License
#
# (C) 2023 Cristóvão Beirão da Cruz e Silva <cbeiraod@cern.ch>
#
# This software is provided 'as-is', without any express or implied
# warranty.  In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.
#############################################################################

#############################################################################
# Modified for ETROC2 I2C testing in jupyter notebooks, Murtaza Safdari
#############################################################################

#############################################################################
# Modified for ETROC2 CERN September 2023 Test Beam, Zach Flowers
#############################################################################

# Imports

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt
import logging
import i2c_gui
import i2c_gui.chips
from i2c_gui.usb_iss_helper import USB_ISS_Helper
from i2c_gui.fpga_eth_helper import FPGA_ETH_Helper
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
# import time
from tqdm import tqdm
# from i2c_gui.chips.etroc2_chip import register_decoding
import os, sys
import multiprocessing
os.chdir(f'/home/{os.getlogin()}/ETROC2/ETROC_DAQ')
import run_script
import importlib
importlib.reload(run_script)
import datetime
import pandas
from pathlib import Path
import subprocess
import sqlite3
from notebook_helpers import *

# Set defaults

In [ ]:
# !!!!!!!!!!!!
# It is very important to correctly set the chip name, this value is stored with the data
chip_names = ["ET2-W36-IP7-12", "ET2-W42-IP5-3"]
chip_fignames = chip_names
chip_figtitles = chip_names

# 'The port name the USB-ISS module is connected to. Default: COM3'
port = "/dev/ttyACM1"
# I2C addresses for the pixel block and WS
chip_addresses = [0x78, 0x68]
ws_addresses = [None, None]

# i2c_gui.__no_connect__ = False  # Set to fake connecting to an ETROC2 device
# i2c_gui.__no_connect_type__ = "echo"  # for actually testing readback
# #i2c_gui.__no_connect_type__ = "check"  # default behaviour

fig_outdir = Path('../ETROC-figures')
fig_outdir = fig_outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
fig_outdir.mkdir(exist_ok=True)
fig_path = str(fig_outdir)

# Make i2c_connection class object

In [ ]:
# i2c_conn = self, port, chip_addresses, chip_names, chip_fc_delays
i2c_conn = i2c_connection(port,chip_addresses,chip_names,[("1","1"), ("1","1")])

# Config chips

### Key is (Disable Pixels, Auto Cal, Chip Peripherals, Basic Peri Reg Check, Pixel Check)

In [ ]:
i2c_conn.config_chips('00011111')

## Visualize the learned Baselines (BL) and Noise Widths (NW)

Note that the NW represents the full width on either side of the BL

In [ ]:
#%%
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# from mpl_toolkits.axes_grid1 import make_axes_locatable
for chip_address, chip_name in zip(chip_addresses,chip_names):
    BL_map_THCal,NW_map_THCal,BL_df = i2c_conn.get_auto_cal_maps(chip_address)
    fig = plt.figure(dpi=200, figsize=(10,10))
    gs = fig.add_gridspec(1,2)

    ax0 = fig.add_subplot(gs[0,0])
    ax0.set_title(f"{chip_name}: BL (DAC LSB)")
    img0 = ax0.imshow(BL_map_THCal, interpolation='none')
    ax0.set_aspect("equal")
    ax0.invert_xaxis()
    ax0.invert_yaxis()
    plt.xticks(range(16), range(16), rotation="vertical")
    plt.yticks(range(16), range(16))
    divider = make_axes_locatable(ax0)
    cax = divider.append_axes('right', size="5%", pad=0.05)
    fig.colorbar(img0, cax=cax, orientation="vertical")

    ax1 = fig.add_subplot(gs[0,1])
    ax1.set_title(f"{chip_name}: NW (DAC LSB)")
    img1 = ax1.imshow(NW_map_THCal, interpolation='none')
    ax1.set_aspect("equal")
    ax1.invert_xaxis()
    ax1.invert_yaxis()
    plt.xticks(range(16), range(16), rotation="vertical")
    plt.yticks(range(16), range(16))
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes('right', size="5%", pad=0.05)
    fig.colorbar(img1, cax=cax, orientation="vertical")

    for x in range(16):
        for y in range(16):
            # if(BL_map_THCal.T[x,y]==0): continue
            ax0.text(x,y,f"{BL_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
            ax1.text(x,y,f"{NW_map_THCal.T[x,y]:.0f}", c="white", size=5, rotation=45, fontweight="bold", ha="center", va="center")
    plt.show()
    # plt.savefig(fig_path+"/BL_NW_"+chip_figname+"_"+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")+".png")

In [ ]:
for chip_address in chip_addresses:
    i2c_conn.onchipL1A(chip_address, comm='00')

In [ ]:
for chip_address in chip_addresses:
    i2c_conn.asyAlignFastcommand(chip_address)

# Define pixels of interest

In [ ]:
row_list = [14, 14, 14, 14]
col_list = [6, 7, 8, 9]
scan_list = list(zip(row_list, col_list))
print(scan_list)

# Enable pixels of Interest

In [ ]:
i2c_conn.enable_pixels_chips(scan_list)

# Run One Time DAQ to Set FPGA Firmware

In [ ]:
# one time run for set fpga firmware
parser = run_script.getOptionParser()
(options, args) = parser.parse_args(args=f"-f --useIPC --hostname 192.168.2.3 -t 20 -o CanBeRemoved -v -w -s 0x0000 -p 0x000f -d 0x1800 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs'))
process.start()

IPC_queue.put('start L1A trigger bit')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('stop L1A trigger bit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

In [ ]:
# one time run for set fpga firmware
parser = run_script.getOptionParser()
(options, args) = parser.parse_args(args=f"-f --useIPC --reset_till_trigger_linked --hostname 192.168.2.3 -t 20 -o CanBeRemoved -v -w -s 0x0000 -p 0x000f -d 0x1800 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, f'main_process_Start_LEDs'))
process.start()

IPC_queue.put('start L1A trigger bit')
while not IPC_queue.empty():
    pass
time.sleep(10)
IPC_queue.put('stop DAQ')
IPC_queue.put('stop L1A trigger bit')
while not IPC_queue.empty():
    pass
IPC_queue.put('allow threads to exit')
process.join()

# Data Acquisition

Make sure the IP address is correct for the FPGA being used. Note that the output is being sent to a folder called "test". User can tune max time of operation of this code (in seconds) using the -t flag

The following cell starts the trains of L1A and Qinj fast commands, each with a different frequency. L1A auccessfully catches the Qinj at the frequency of the beat pattern, with a period equal to the LCM of the two periods

This cell allows the read process to timeout, and allows the process to exit before the "-t" time in case the readout processes have been waiting for more than 30sec and then stops the trains of L1A and Qinj fast commands

In [ ]:
parser = run_script.getOptionParser()
# (options, args) = parser.parse_args(args=f"-f --useIPC --hostname 192.168.2.3 -t 60 -o MultiBoard -v -w --compressed_translation -s 0x000C -p 0x000f --clear_fifo".split())
(options, args) = parser.parse_args(args=f"--useIPC --reset_till_linked --hostname 192.168.2.3 -t 20 -o MultiBoard -v -w --compressed_translation -s 0x0000 -p 0x000f -d 0x1800 --clear_fifo".split())
IPC_queue = multiprocessing.Queue()
process = multiprocessing.Process(target=run_script.main_process, args=(IPC_queue, options, "main_process"))
process.start()
IPC_queue.put('start L1A trigger bit') ## Trigger bit
while not IPC_queue.empty():
    pass
time.sleep(5)
IPC_queue.put('stop DAQ')
IPC_queue.put('stop L1A trigger bit') ## Trigger bit
IPC_queue.put('allow threads to exit')
while not IPC_queue.empty():
    pass
process.join()

After ending the process, one can inspect the stored raw binary and translated files to check for data integrity

# Disconnect I2C Device

In [ ]:
del i2c_conn

# Store BL, NW dataframe for later use

In [ ]:
outdir = Path('/run/media/daq/T7')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (chip_name+"_BaselineAt_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
BL_df.to_csv(outfile, index=False)

In [ ]:
### Store BL, NW dataframe for later use
outdir = Path('/run/media/daq/T7')
outdir = outdir / (datetime.date.today().isoformat() + '_Array_Test_Results')
outdir.mkdir(exist_ok=True)
outfile = outdir / (chip_name+"_BaselineAt_" + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M") + ".csv")
BL_df.to_csv(outfile, index=False)
new_columns = {
    'note': f'{chip_figname}',
}

for col in new_columns:
    BL_df[col] = new_columns[col]

outdir = Path('../ETROC-Data')
outfile = outdir / 'BaselineHistory.sqlite'

init_cmd = [
    'cd ' + str(outdir.resolve()),
    'git stash -u',
    'git pull',
]
end_cmd = [
    'cd ' + str(outdir.resolve()),
    'git add BaselineHistory.sqlite',
    'git commit -m "Added new history entry"',
    'git push',
    'git stash pop',
    'git stash clear',
]
init_cmd = [x + '\n' for x in init_cmd]
end_cmd  = [x + '\n' for x in end_cmd]

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

# for cmd in init_cmd:
#     p.stdin.write(cmd + "\n")
# p.stdin.close()
# p.wait()

# print(p.stdout.read())

with sqlite3.connect(outfile) as sqlconn:
    BL_df.to_sql('baselines', sqlconn, if_exists='append', index=False)

p = subprocess.Popen(
    '/bin/bash',
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    encoding="utf-8",
    )

# for cmd in end_cmd:
#     p.stdin.write(cmd + "\n")
# p.stdin.close()
# p.wait()

# p.stdin.close()

# print(p.stdout.read())
